In [1]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from openpyxl import Workbook
from konlpy.tag import Okt
from collections import Counter
import time

options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--headless")

driver = webdriver.Chrome(options=options)

url = "https://www.kurly.com/goods/1000078028"
test = driver.get(url)
time.sleep(5)

soup = BeautifulSoup(driver.page_source, "html.parser")
reviews = soup.select("p.css-y49dcn.e36z05c13")
review_lists = [review.get_text().strip() for review in reviews]

driver.quit()

wb = Workbook()
ws = wb.active
ws.title = "Kurly Reviews"
ws.append(["리뷰내용"])

for item in review_lists :
    ws.append([item])

okt = Okt()
all_text = " ".join(review_lists)
nouns = okt.nouns(all_text)
filtered_nouns = [n for n in nouns if len(n) > 1]
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("Keyword Frequency")
ws_keywords.append(["키워드", "빈도수"])
for word, freq in count.most_common(30) :
    ws_keywords.append([word, freq])

wb.save("kurly_reviews.xlsx")
print("엑셀 저장 완료 : kurly_reviews.xlsx")

The chromedriver version (137.0.7151.104) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (138.0.7204.158); currently, chromedriver 138.0.7204.157 is recommended for chrome 138.*, so it is advised to delete the driver in PATH and retry


엑셀 저장 완료 : kurly_reviews.xlsx


In [20]:
from openpyxl import load_workbook
from konlpy.tag import Okt
from collections import Counter

wb = load_workbook("연관키워드 20250722 1448.xlsx")
ws = wb.active

texts = [row[0] for row in ws.iter_rows(min_row = 2, max_col = 1, values_only = True) if row[0]]

okt = Okt()
all_text = " ".join(texts)
nouns = okt.nouns(all_text)
filtered_nouns = [n for n in nouns if len(n) > 1]
count = Counter(filtered_nouns)

ws_keywords = wb.create_sheet("키워드 전체분석")
ws_keywords.append(["명사", "빈도수"])
for word, freq in sorted(count.items(), key=lambda x: x[1], reverse = True) :
    ws_keywords.append([word, freq])

wb.save("연관키워드_분석결과.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨


In [21]:
from openpyxl import load_workbook
from konlpy.tag import Okt
from collections import Counter
import re

wb = load_workbook("연관키워드 20250722 1448.xlsx")
ws = wb.active

texts = [row[0] for row in ws.iter_rows(min_row = 2, max_col = 1, values_only = True) if row[0]]

okt = Okt()
all_text = " ".join(texts)

nouns_tokens = okt.nouns(all_text)

regex_tokens = re.findall(r'[A-Za-z]*\d+[A-Za-z]+|\d+[A-Za-z]*', all_text)

valid_one_letter = {"금", "은", "동"}
stopwords = {"사용", "정도", "조금", "제품", "때문", "생각", "만족", "구매", "다음"}

combined_tokens = nouns_tokens + regex_tokens
filtered_tokens = [t for t in combined_tokens if (len(t) > 1 or t in valid_one_letter) and t not in stopwords]

count = Counter(filtered_tokens)

ws_keywords = wb.create_sheet("키워드 전체분석")
ws_keywords.append(["명사", "빈도수"])
for word, freq in sorted(count.items(), key=lambda x: x[1], reverse = True) :
    ws_keywords.append([word, freq])

wb.save("연관키워드_분석결과.xlsx")
print("완료 : 전체 키워드 분석 결과 저장됨")

완료 : 전체 키워드 분석 결과 저장됨
